## Combining and Merging datasets



-   `merge` connects rows in DataFrames based on one or more keys
-   `concat` stacks together objects along an axis
-   `combine_first` splices together overlapping data to fill in missing values



### Database-style joins



Let's start with a simple example



In [2]:
import pandas as pd
import numpy as np
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

In [3]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df2

,key,data2
0,a,0
1,b,1
2,d,2


Try joining the two DataFrames and explain how the different types work

-   `inner`
-   `left`
-   `right`
-   `outer`



In [11]:
pd.merge(df1, df2, how='left')

,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


![img](images/merge.png)



Let's try joining the two DataFrames given below

In [12]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [17]:
pd.merge(left2, right2, left_index=True, right_index=True, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


### Concatenating pandas objects



The `concat` function provides a consistent way of "stacking" data



In [18]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

The default operation is to just "glue" together the values and indexes



In [19]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

Can also be done along columns



In [20]:
pd.concat([s1, s2, s3], axis=1)

/usr/local/miniconda3/envs/cee597/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


What if there are common labels?



In [21]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

Can you replicate inner and outer join using `concat`?



In [26]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


-   A potential issue is that the concatenated pieces are not identifiable in the result
-   Instead we could create a hierarchical index on the concatenation axis



In [1]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

In [1]:
result.unstack()

### Combining data with overlap



What if we had two datasets whose indexes overlap in full or part?



In [1]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
                index=['f', 'e', 'd', 'c', 'b', 'a'])
a

In [1]:
b = pd.Series(np.arange(len(a), dtype=np.float64),
                index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
b

Can you create an array using `np.where`  that will fill-in missing data with values from b?



In [1]:
b.combine_first(a)

## Data aggregation



-   After loading, cleaning and merging a dataset, we may need to compute group statistics or *pivot* tables
-   pandas provides a flexible `groupby` interface that allows us to manipulate datasets in a natural way



### Groupby mechanics



`split-apply-combine` is at the core of these operations

![img](images/splitapplycombine.png)



In [27]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                     'key2' : ['one', 'two', 'one', 'two', 'one'],
                     'data1' : np.random.randn(5),
                     'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,2.293463,0.884978
1,a,two,0.023360,-1.288242
2,b,one,0.436572,0.948291
3,b,two,0.943837,-0.428013
4,a,one,0.698859,-1.134483


Suppose we wanted to calculate the means of the *data1* column grouping by the *key1* labels



In [28]:
grouped = df['data1'].groupby(df['key1'])
grouped

What is this object? How can we calculate the means?



In [31]:
grouped.std()

key1
a    1.16565
b    0.35869
Name: data1, dtype: float64

What if we passed multiple arrays as a list?



In [32]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     1.496161
      two     0.023360
b     one     0.436572
      two     0.943837
Name: data1, dtype: float64

The grouping keys can be any sequence



In [33]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.023360
            2006    0.436572
Ohio        2005    1.618650
            2006    0.698859
Name: data1, dtype: float64

### Iterating over groups



The GroupBy object supports iteration, generating a sequence of 2-tuples containing
the group name along with the chunk of data



In [34]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  2.293463  0.884978
1    a  two  0.023360 -1.288242
4    a  one  0.698859 -1.134483
b
  key1 key2     data1     data2
2    b  one  0.436572  0.948291
3    b  two  0.943837 -0.428013


### Grouping with Dictionaries and Series



Grouping can be performed with different forms of sequences



In [1]:
people = pd.DataFrame(np.random.randn(5, 5)
                        columns=['a', 'b', 'c', 'd', 'e'],
                        index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan
people

If we have a color corresponding to each letter a-f, we can calculate the sum for each name grouped by color



In [1]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f' : 'orange'}
by_column = people.groupby(mapping, axis=1)
by_column.sum()

Same functionality holds for Series. Can you convert the mapping into a Series and then calculate the number of values for each name and color?



### Grouping with functions



Using Python functions is a more generic way of defining a group mapping compared
with a dict or Series



In [1]:
people

Suppose we wanted to group by the length of each name and calculate the sum



In [1]:
people.groupby(len).sum()

We can also mix functions with any other grouping instance



In [1]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

## Homework



Download the ensemble streamflow forecasts from [https://www.cnrfc.noaa.gov/ensembleProductCSV.php>](https://www.cnrfc.noaa.gov/ensembleProductCSV.php>)for the Klamath River and load them into a DataFrame. Each column corresponds to a forecast point and an ensemble member, e.g. GERO3.2 corresponds to the second member for the GERO3 station.

1.  Calculate the mean daily flow for each forecast point.
2.  What is the minimum and maximum values from the ensemble for each forecast point?
3.  Print the ensemble statistics for each forecast point.
4.  Find the forecast points that had the 5 largest flow volumes as predicted by their first ensemble member.
5.  Create a new DataFrame that will have each column correspond to the mean daily flow Series for each forecast point.
6.  Create a new DataFrame that will have a column for each forecast point and values for each day that correspond to the fraction of the ensemble that is higher than the average flow (from all days) for that forecast point.
7.  Now download the data for the Tulare River and after creating a similar DataFrame like the above, merge the two tables with each row corresponding to a date.

